<h2><u>STEP: 01</u></h2>



<h2><u>STEP: 02</u></h2>

In [27]:
import os
import pandas as pd
from string import punctuation

# libraries importing from scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

# natural language toolkit packages
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer, TreebankWordTokenizer
from nltk.stem.snowball import  EnglishStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

In [28]:
# method to get all the text files from the directory to a list
def read_class_files(dir):
    fl = []
    for roots, dirs, files in os.walk(dir):
        for file in files:
            fl.append(os.path.join(roots, file))
    return fl

In [29]:
files = read_class_files('./')

In [30]:
# method to categorize and read the files
comp_fl = []
rec_fl = []
sci_fl = []
talk_fl = []

In [31]:
def categorize_files(files):
    for file in files:
        if "comp" in file:
            comp_fl.append(file)
        elif "rec" in file:
            rec_fl.append(file)
        elif "sci" in file:
            sci_fl.append(file)
        elif "talk" in file:
            talk_fl.append(file)

In [32]:
categorize_files(files)

In [33]:
print("comp  "+str(len(comp_fl)))
print("sci "+str(len(sci_fl)))
print("talk  "+str(len(talk_fl)))
print("rec "+str(len(rec_fl)))

comp  4891
sci 3952
talk  3253
rec 3979


In [34]:
comp_train,comp_test = train_test_split(comp_fl, train_size=0.7,test_size=0.3, random_state=42)
rec_train, rec_test = train_test_split(rec_fl, train_size=0.7,test_size=0.3, random_state=42)
sci_train, sci_test = train_test_split(sci_fl, train_size=0.7,test_size=0.3, random_state=42)
talk_train,talk_test = train_test_split(talk_fl, train_size=0.7,test_size=0.3, random_state=42)

In [35]:
All_train_data_file =[ comp_train, rec_train, sci_train, sci_train]
All_test_data_file =[ comp_test, rec_test, sci_test, sci_test]

In [36]:
# get the set of stop words
_stop_words = set(stopwords.words('english') + list(punctuation))

# initialize english stemmer
stemmer = EnglishStemmer()

# initialize word net lemmatizer
lemmatizer = WordNetLemmatizer()

# word list used in english language
_word_list = set([word for word in wn.words(lang='eng')])

In [37]:
#get unique words
def process_files(data_list):
    word_set = []
    for docs in data_list:
        for doc in docs:
            words = []
            for word in word_tokenize(doc):
                word = stemmer.stem(word)
                word = lemmatizer.lemmatize(word)
                words += [word]
            word_set += [word for word in words if word not in _stop_words if word in _word_list if word.isalpha()]
    return list(set(word_set))

In [38]:
voc_train = process_files (All_train_data_file)

In [39]:
print(voc_train)

[]


In [ ]:
clean_train_comp = process_files(comp_train)

In [25]:
print(voc_train)

[]


In [21]:
def vectorize_csv(cleaned_data, name, vocabulary_set):
    vectorizer = CountVectorizer(max_features=500,
stop_words=_stop_words, vocabulary=vocabulary_set)
    x = vectorizer.fit_transform(cleaned_data)
    y = vectorizer.get_feature_names()
    
    df = pd.DataFrame(data=x.toarray(), columns=y)
    
    df.to_csv(name+'_freq.csv')
    
    print(name+'_freq.csv successfully created!')
    return df

In [24]:
vectorize_csv (clean_train_comp, 'train_comp', voc_train)

ValueError: empty vocabulary passed to fit